# APR 2021 Entregable 1: MDPs y Programación Dinámica

## Total: 100 puntos

Temas tratados en este entregable:



*   Markov Decission Processes (MDPs)
    * Cálculo de probabilidades de transición en T pasos
    * Ejemplo: "Student MDP"

*   Programación Dinámica:
    * Policy Improvement Theorem
    * Policy Evaluation
    * Policy Iteration
    * Value Iteration



Se pide responder algunas preguntas y completar porciones de código.

Lectura sugerida: Sutton & Barto 2020 Caps. 3 y 4 [(link)](http://incompleteideas.net/book/RLbook2020.pdf)

In [ ]:
# Paquetes a importar
import numpy as np
from matplotlib import pyplot as plt

# 1. Probabilidades de transición (25p)
 

## 1.1
Considerar un MDP $(\mathcal{S}, \mathcal{A}, \mathcal{R}, \mathcal{P})$, donde $\mathcal{S}$ y $\mathcal{A}$ son los espacios de estados y acciones, discretos, con cardinalidad $N$ y $M$ respectivamente.

Escribir la probabilidad de llegar al estado $s'$ luego de $T$ pasos, arrancando del estado $s$ y siguiendo la politica $\pi(a|s)$

\begin{equation}
P(S_T = s^\prime | S_0 = s),
\end{equation}
donde $S_T$ y $S_0$ son variables aleatorias representando el estado del sistema en tiempo $T$ y $0$. Esta es una generalizacion de las transiciones de uno y dos pasos vista en el curso (slides 21-23).




*Sugerencia*: escribir para $T=1$ y $T=2$ y usar inducción. 

**Tu respuesta:**



## 1.2

Una forma más concisa de escribir las probabilidades de transición en varios pasos se obtiene usando matrices. Definamos la matriz $\mathbf{P} \in \mathbb{R}^{N\times N}$ cuyas entradas $(~\mathbf{P}~)_{ij}=P\left(S_{t+1}=s_j | S_{t}=s_i\right)$. Diremos que esta es la *matriz de transición* del MDP inducida por la política $\pi$.

La matriz $\mathbf{P}$ es una *matriz estocástica*, con las siguientes propiedades:

* Sus entradas son no negativas
* Sus entradas son menores o iguales a 1
* La suma de los valores de cada fila da 1 (¿por qué?)

Consideremos el vector $\mathbf{p}_0 \in \mathbb{R}^N$ de entradas $(\mathbf{p}_0)_i = P(S_o=s_i)$. Este vector representa la distribución de probabilidad del estado inicial, es decir, la probabilidad de que el MDP arranque en cada uno de los estados.

Se pide:

1) Obtener una expresión para $\mathbf{p}_T \in \mathbb{R}^N$, vector con la distribución de probabilidad luego de $T$ pasos, en función de $\mathbf{P}$ y $\mathbf{p}_0$.

2) Si $\mathbf{p}_0$ contiene un $1$ en el $j$-ésimo índice y sus demás entradas son nulas, ¿qué significado tiene aplicar el resultado de la parte anterior?

**Tu respuesta:**

## 1.3

Vamos a trabajar sobre el ejemplo del estudiante (slide 50). Los estados posibles para el estudiante son $\texttt{happy (H)}$ o $\texttt{sad (S)}$.
En cada estado se puede tomar dos acciones: estudiar o tomar cerveza.

Consideraremos que existen dos tipos de estudiantes (o políticas):  aquellos que siempre estudian y aquellos que siempre toman cerveza. 

<img src="https://drive.google.com/uc?id=1OE9yyEKlzo2QOXo4_loxcK0zM3cBYgzV" width=500px />

En esta parte se pide escribir un programa que determine la probabilidad de estar en cada estado luego de tantos pasos, siguiendo cualquiera de las dos politicas.

### Setup

In [ ]:
# Las siguientes clases implementan el MDP y una politica deterministica
# Pueden servir como base para resolver lo que se pide mas abajo


class StudentMDP(object):
  """
  Clase básica que implementa el MDP. 
  Usar get_data() para obtener probabilidades de transición y rewards.
  """

  ACTION_DRINK = 0
  ACTION_STUDY = 1
  STATE_HAPPY = 0
  STATE_SAD = 1

  def __init__(self):
      self.data = [[[[], []], [[], []]], [[[], []], [[], []]]]
      self.data[StudentMDP.STATE_HAPPY][StudentMDP.ACTION_DRINK][StudentMDP.STATE_HAPPY] = (1. , 10.)
      self.data[StudentMDP.STATE_HAPPY][StudentMDP.ACTION_DRINK][StudentMDP.STATE_SAD] = (0., 0.)
      self.data[StudentMDP.STATE_HAPPY][StudentMDP.ACTION_STUDY][StudentMDP.STATE_HAPPY] = (0.2, -10.)
      self.data[StudentMDP.STATE_HAPPY][StudentMDP.ACTION_STUDY][StudentMDP.STATE_SAD] = (0.8, -10.)
      self.data[StudentMDP.STATE_SAD][StudentMDP.ACTION_DRINK][StudentMDP.STATE_HAPPY] = (0.8, 40.)
      self.data[StudentMDP.STATE_SAD][StudentMDP.ACTION_DRINK][StudentMDP.STATE_SAD] = (0.2, 40)
      self.data[StudentMDP.STATE_SAD][StudentMDP.ACTION_STUDY][StudentMDP.STATE_HAPPY] = (0.8, 20)
      self.data[StudentMDP.STATE_SAD][StudentMDP.ACTION_STUDY][StudentMDP.STATE_SAD] = (0.2, 20)

  def get_data(self, state, action, state_prime):            
    # Calcula la probabilidad de transición y el reward obtenido al ir de 'state'
    # a 'state_prime' bajo la acción 'action'.
    # Ej.: p, r = get_data(StudentMDP.STATE_HAPPY, StudentMDP.ACTION_DRINK, StudentMDP.STATE_SAD)      

      if (state < 0 or state >= self.get_num_states()):
          raise Exception("Invalid state")
      if (state_prime < 0 or state_prime >= self.get_num_states()):
          raise Exception("Invalid state_prime")
      if (action < 0 or action >= self.get_num_actions()):
          raise Exception("Invalid action")
      return self.data[state][action][state_prime]

  def get_num_states(self):
    # Cantidad de estados
      return 2

  def get_num_actions(self):
    # Cantidad de acciones
      return 2


class DeterministicPolicy(object):
  """
  Clase que describe una política determinística. Ejemplo de uso: 
  
  policy = DeterministicPolicy(2, 2)
  for state in [StudentMDP.STATE_HAPPY, StudentMDP.STATE_SAD]:
    policy.set_action(state, StudentMDP.ACTION_DRINK)  
  """

  def __init__(self, num_states, num_actions):
      self.num_states = num_states
      self.num_actions = num_actions        
      self.state_action_map = np.zeros(num_states, dtype=np.int)

  def get_action(self, state):
      if (state < 0 or state >= self.num_states):
          raise Exception("Invalid state")
      return self.state_action_map[state]

  def set_action(self, state, action):
      if (state < 0 or state >= self.num_states):
          raise Exception("Invalid state")
      if (action < 0 or action >= self.num_actions):
          raise Exception("Invalid action")
      self.state_action_map[state] = action

### 1.3.1 
Escribir una función que calcule la probabilidad de estar en $s'$ luego de $T$ pasos arrancando desde $s$, siguiendo cualquiera de estas dos politicas.


In [ ]:
# ------------
# Tu respuesta
# ------------

### 1.3.2

Usando esa función, calcular para cada estudiante las siguientes probabilidades:



1.   $P(S_1 =\texttt{happy} \mid S_0 = \texttt{happy})$
2.   $P(S_1 = \texttt{happy} \mid S_0 = \texttt{sad})$
3.   $P(S_2 = \texttt{happy} \mid S_0 = \texttt{happy})$
4.   $P(S_2 = \texttt{happy} \mid S_0 = \texttt{sad})$
5.   $P(S_3 = \texttt{happy} \mid S_0 = \texttt{happy})$
6.   $P(S_3 = \texttt{happy} \mid S_0 = \texttt{sad})$

In [ ]:
# ------------
# Tu respuesta
# ------------

Calcular $P(S_2 = \texttt{happy} \mid S_0 = \texttt{sad})$ usando la expresión obtenida en (1.1). Comparar con los resultados experimentales.

**Tu respuesta:**


---
# 2. Policy Improvement (25p)

Dado un MDP $(\mathcal{S}, \mathcal{A}, \mathcal{R}, \mathcal{P})$ con factor de descuento $\gamma$ y una política $\pi(a|s)$, definimos la q-function como


\begin{equation}
q_\pi(s, a) = \mathbb{E}[R_t + \gamma v_\pi(S_{t+1}) | S_t = s, A_t = a].
\end{equation}


Consideremos una nueva política $\pi^\prime(a|s)$ tal que para todo $s\in\mathcal{S}$ elige $a =\text{argmax}_{a\in\mathcal{A}} q_\pi(s,a)$ una vez y luego sigue $\pi(a|s)$. 
El Policy Improvement Theorem dice que

\begin{equation}
v_{\pi^\prime} (s) \geq v_{\pi} (s),  \forall s\in\mathcal{S}.
\end{equation}

Vamos a demostrar esto en varios pasos. La idea de la prueba es arrancar desde un estado arbitrario $s\in\mathcal{S}$, dar un paso usando $\pi^\prime(a|s)$ y luego usar $\pi(a|s)$. Esto debería dar un retorno esperado mayor que usar $\pi$ siempre. Luego, consideramos dar los dos primeros pasos con $\pi^\prime$ y luego usar $\pi$, y mostraremos que el retorno es mejor, y así sucesivamente. 







## 2.1 

Demostrar $$\mathbb{E}_{a \sim \pi^\prime}[q_\pi(s, a)] \geq v_\pi(s).$$

Esto quiere decir que el retorno esperado dando el primer paso con $\pi^\prime$ y luego seguir $\pi$ debe ser al menos tan bueno cómo seguir $\pi$ siempre.

**Tu respuesta**

## 2.2

Definimos por conveniencia $Q_1 := \mathbb{E}_{a \sim \pi^\prime}[q_\pi(s, a)] $.
Consideremos ahora que la política  $\pi^\prime(a|s)$ aplica $a=\text{argmax}_{a\in\mathcal{A}}q(s,a)$ dos veces y luego sigue $\pi(a|s)$.

Demostrar $$Q_2 := \mathbb{E}_{\pi^\prime}[R_t + \gamma R_{t+1} + \gamma^2 v_\pi(S_{t+2})] \geq Q_1.$$



## 2.3
Definiendo análogamente $Q_l$ for all $l\geq 1$ usar un razonamiento similar al de (2.2) y demostrar
$$Q_{l+1} \geq Q_{l}.$$

**Tu respuesta**



## 2.4
Usando esto podemos concluir que $Q_\infty \geq v_\pi(s)$. ¿Qué es $Q_\infty$? 

**Tu respuesta**



Un par de comentarios:

1.   Todo el razonamiento de la prueba fue usando un estado $s\in\mathcal{S}$ fijo. Sin embargo, podemos construir $\pi^\prime$ de cualquier forma. En vez de elegir la mejor acción en un único estado, podemos hacerlo para todos los estados, ¡y la prueba sigue funcionando!

2.   El enunciado del Policy Improvement Theorem es en realidad un poco más genérico:

 Dadas dos politicas $\pi(a|s)$ y $\pi^\prime(a|s)$, si $\mathbb{E}_{a \sim \pi^\prime}[q_\pi(s, a)] \geq v_\pi(s), \forall s \in \mathcal{S}$, entonces

\begin{equation*}
v_{\pi^\prime}(s) \geq v_\pi(s), \forall s\in\mathcal{S}.
\end{equation*}

---

# 3. Policy Iteration (25p)

Policy Iteration (Iteración de Política) es un algoritmo que aprovecha el Policy Improvement Theorem. La idea básica es comenzar con una cierta política $\pi_0$, evaluar esa política calculando $v_{\pi_0}(s), \forall s\in\mathcal{S}$. 
Luego construye una política $\pi_1$ que resulta de tomar la mejor acción respecto a $q_{\pi_0}(s,a)$ para cada estado; luego se evalúa $v_{\pi_1}(s) \forall s\in\mathcal{S}$ y se repite este procedimiento para generar $\pi_2, \pi_3, \ldots$ hasta que $\pi$ converja.

El Policy Improvement Theorem visto en la sección anterior garantiza que en cada paso la nueva política mejore el valor $v(\cdot)$ en al menos un estado, salvo cuando la política original ya es optimal.

Resumimos el algoritmo más abajo, tomado de [Sutton & Barto 2020, pág. 80]

<img src="https://drive.google.com/uc?id=198ZqpPx9j1dgJHozbEdsHLxz8O3_tQ_l" width=800px/>


## 3.1

En esta parte vamos a implementar Policy Iteration. Separamos la tarea en tres funciones: la primera implementa Policy Evaluation, la segunda Policy Improvement, la tercera chequea si dos politicas son iguales.

Completar cada una de las siguientes funciones.






 

In [ ]:

# TAREA: Completar esta función.
# Esta función implementa Policy Evaluation. Toma un StudentMDP
# y una política para ser evaluada. Devuelve un numpy array de tamaño (num_states,)
# que contiene la función de valor en cada estado

def PolicyEvaluation(mdp, policy, gamma=.9):

  N = mdp.get_num_states()
  M = mdp.get_num_actions()
  
  value = np.zeros(N)
  transition_matrix = np.zeros((N, N))

  for state in range(N):
    action = policy.get_action(state)
    for next_state in range(N):
      # --------------------------------
      # Completar: calcular value[state]
      # --------------------------------
      pass
  return value


# TAREA: Completar esta función.
# Esta función implementa Policy Improvement. Toma un MDP y una función de valor
# (obtenida aplicando PolicyEvaluation()), calcula la q-function para cada estado
# y devuelve una nueva política, que actúa de forma greedy respecto a q.

def PolicyImprovement(mdp, value_func, gamma=.9):

  N = mdp.get_num_states()
  M = mdp.get_num_actions()
  policy = DeterministicPolicy(N, M)

  for state in range(N):
    q_values = np.zeros(M)
    for action in range(M):
      # ------------------------------------
      # Completar: calcular q_values[action]
      # ------------------------------------    
      pass

    # --------------------------------        
    # Completar: modificar la política
    # --------------------------------
      
  return policy
    

# TAREA: Completar esta función
# Esta función toma dos políticas old_policy y new_policy,
# siendo ambas del tipo DeterministicPolicy().
# devuelve True o False dependiendo de si PolicyIteration debe terminar o no.

def ShouldTerminate(mdp, old_policy, new_policy):
  # ---------
  # Completar
  # ---------
  return     



def PolicyIteration(mdp, initial_policy, gamma=.9):
  policy = initial_policy
  
  idx = 0
  while True:
    value_func = PolicyEvaluation(mdp, policy, gamma)
    new_policy = PolicyImprovement(mdp, value_func, gamma)

    print("Iteración " + str(idx) + ": valor = " + str(value_func.T))

    should_terminate = ShouldTerminate(mdp, policy, new_policy)
    if (should_terminate):
        break
    policy = new_policy
    idx += 1

  print("Pronto: " + str(value_func.T))
  return policy

## 3.2

Aplicar Policy Iteration en el ejemplo del ejercicio 1. Presentar tres gráficas:



1.   Función de valor $v(s)$  para cada estado ($\texttt{happy}$, $\texttt{sad})$ en función de $k$, siendo $k$ la cantidad de veces que se ejecuta Policy Evaluation.
2.   Tabla con la política $\pi(a|s)$ al converger.



In [ ]:
# ------------
# Tu respuesta
# ------------

Desde un punto de vista computacional, ¿qué desventajas le encontrás a este algoritmo?

**Tu respuesta**

# 4. Value iteration (25p)


Una de las desventajas de Policy Iteration es que en cada paso evalúa la política actual. Esto a la larga puede ser computacionalmente exigente, además de que no vale la pena evaluar perfectamente una política que en el siguiente paso se va a modificar.

Una alternativa es actualizar alternadamente la función de valor y la política. Este algoritmo se llama Value Iteration (Iteración de Valor) [Sutton & Barto 2020, pág. 83]:




<img src="https://drive.google.com/uc?id=19l-A5QLwpS1WArveVtPCc-4Vi0YcP8rZ" width=800px/>

## 4.1
Implementar el algoritmo de Value Iteration




In [ ]:
 def ValueIteration(mdp, initial_policy, gamma=.9):
  # ---------
  # Completar
  # ---------
  return policy

## 4.2

 Aplicar el algoritmo para el MDP del Ejercicio 1. Mostrar:



1.   Evolución de la función de valor para cada estado, en función del número de iteraciones del loop de más afuera 
2.   Tabla con la política $\pi(a|s)$ al converger.

In [ ]:
# ------------
# Tu respuesta
# ------------